In [1]:
import ee
import geemap
import math
import itertools
import pandas as pd

In [2]:
#Initialize Google Earth Engine
#ee.Authenticate() just needed the 1st time
ee.Initialize()

In [3]:
#Data loads

#loads feature collection data from Google Earth Engine - We can also upload other feature collections
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))
sd_county = counties.filter(ee.Filter.eq('NAME', 'San Diego'))
so_cal = counties.filter(ee.Filter.inList('GEOID',['06073','06037','06059']))

In [4]:
#loads cities data - Not in current use

zipcodes = ee.FeatureCollection("projects/california-lawn-detection/assets/tl_2021_us_zcta520") 

In [5]:
def get_images(param_dict):
    source_image_collection = params['source_image_collection']
    years = param_dict['years']
    counties = param_dict['counties']

    image_names = []
    images = []

    combos = list(itertools.product(years, counties.keys()))
    for i in combos:
        year = str(i[0])
        county = i[1]

        image_name = str(i[0])+'_'+i[1]
        image_names.append(image_name)

        image = ee.ImageCollection(source_image_collection)\
                                .filterDate(f'{year}-04-01', f'{year}-06-30')\
                                .select(['LST_Day_1km', 'LST_Night_1km'])\
                                .median().clip(counties[county])
        images.append(image)
           
    return dict(zip(image_names, images))        


In [6]:
params = {
        'source_image_collection' : 'MODIS/006/MOD11A2',
        'years' : [2010,2012,2014,2016,2018,2020],
        'counties': {'so_cal': so_cal}
         }

images = get_images(params)
images

{'2010_so_cal': <ee.image.Image at 0x1b31fb26910>,
 '2012_so_cal': <ee.image.Image at 0x1b320d687f0>,
 '2014_so_cal': <ee.image.Image at 0x1b320d5ba90>,
 '2016_so_cal': <ee.image.Image at 0x1b320d5bcd0>,
 '2018_so_cal': <ee.image.Image at 0x1b37e4b28b0>,
 '2020_so_cal': <ee.image.Image at 0x1b320d511f0>}

In [7]:
image_collection_2020_modis = (
    ee.ImageCollection('MODIS/006/MOD11A2')
    .filterDate('2020-04-01', '2020-06-30')
    .select(['LST_Day_1km', 'LST_Night_1km'])
)

image_2020_modis = image_collection_2020_modis.median()

image_2020_modis_clipped = image_2020_modis.clip(sd_county)

In [8]:
def temperature_calculation(modis_image, shape, pixel_scale):
    
    reducers = ee.Reducer.mean().combine(ee.Reducer.min(),sharedInputs=True).combine(ee.Reducer.max(),sharedInputs=True)
    
    mean_temperature = modis_image.reduceRegion(
        reducer = reducers,
        geometry = shape.geometry(), 
        scale = pixel_scale, 
        maxPixels = 1e13)
    
    lst_day_mean = mean_temperature.getInfo().get('LST_Day_1km_mean') * .02 - 273
    lst_day_max = mean_temperature.getInfo().get('LST_Day_1km_max') * .02 - 273
    lst_day_min = mean_temperature.getInfo().get('LST_Day_1km_min') * .02 - 273
    lst_night_mean = mean_temperature.getInfo().get('LST_Night_1km_mean') * .02 - 273
    lst_night_max = mean_temperature.getInfo().get('LST_Night_1km_max') * .02 - 273
    lst_night_min = mean_temperature.getInfo().get('LST_Night_1km_min') * .02 - 273
    
    return lst_day_mean, lst_day_max, lst_day_min, lst_night_mean, lst_night_max, lst_night_min

In [9]:
year_list = [2010, 2012, 2014, 2016, 2018, 2020]

# zipcode_list = ['90094', '90274', '90272', '93563', '90077','90732','90013', '90021', '90058', '90007', '90014']

zipcode_list = ['92130','92131','92129','92127','92121','92128','92106','92126','92124','92120','92119','92122','92123','92139',
                '92117','92114','92109','92154','92111','92107','92108','92110','92103','92116','92115','92104','92102','92105','92113','92101']


In [10]:
keys = {'year','polygon',
        'lst_day_mean','lst_day_max', 'lst_day_min', 
        'lst_night_mean', 'lst_night_max','lst_night_min'}

year_list = []
polygon_list = []
lst_day_mean = []
lst_day_max = []
lst_day_min = []
lst_night_mean = []
lst_night_max = []
lst_night_min = []
    
    
for i in zipcode_list:
    for j in list(images.items()):
        im = j[1]
        name = j[0]
        
        polygon_list.append(i)  
        year_list.append(j[0][:4])  
        
        #start = time()
        polygon = zipcodes.filter(ee.Filter.eq('GEOID20', i))
        temperature = temperature_calculation(im, polygon, 1000)
        
        temperature = list(temperature)
        lst_day_mean.append(temperature[0]) 
        lst_day_max.append(temperature[1])
        lst_day_min.append(temperature[2])
        lst_night_mean.append(temperature[3])
        lst_night_max.append(temperature[4])
        lst_night_min.append(temperature[5])

              

In [11]:
df_list = list(zip(polygon_list, year_list,lst_day_mean,lst_day_max,lst_day_min,lst_night_mean,lst_night_max,lst_night_min))

In [12]:
df = pd.DataFrame(df_list, columns = ['polygon','year',
        'lst_day_mean','lst_day_max', 'lst_day_min', 
        'lst_night_mean', 'lst_night_max','lst_night_min'])

In [13]:
df

,polygon,year,lst_day_mean,lst_day_max,lst_day_min,lst_night_mean,lst_night_max,lst_night_min
0,92130,2010,30.493352,33.74,23.56,12.746009,14.02,11.70
1,92130,2012,31.008651,35.30,22.94,13.995664,15.52,12.58
2,92130,2014,33.890284,38.40,27.46,15.721636,16.92,14.18
3,92130,2016,30.804146,34.06,24.48,13.757657,15.30,12.46
4,92130,2018,31.491457,35.66,24.70,13.621281,14.82,12.00
...,...,...,...,...,...,...,...,...
175,92101,2012,31.798712,35.68,22.88,15.969636,16.82,14.12
176,92101,2014,33.552172,37.38,24.71,17.579715,18.26,14.26
177,92101,2016,30.887026,35.01,20.98,15.095548,16.68,13.84
178,92101,2018,31.790502,35.42,23.32,15.102751,16.00,13.72


In [14]:
df.to_csv('sd_microclimate.csv')